In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [2]:
#############
# Settings
#############

# Hyperparameters
random_seed = 1
learning_rate = 0.001
batch_size = 128
num_epochs = 10

# Architecture
num_features = 32*32
num_classes = 10

# Other
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
grayscale = False

In [3]:
################
# CIFAR 10
################

train_dataset = datasets.CIFAR10(root = "data",
                                 train = True,
                                 transform = transforms.ToTensor(), # scales it to 0-1 range
                                 download = True
                                )

test_dataset = datasets.CIFAR10(root = "data",
                                train = False,
                                transform = transforms.ToTensor()
                               )

train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, num_workers = 8, shuffle = True)

test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, num_workers = 8, shuffle = False)

# Checking the dataset
for images, labels in train_loader:
    print("Image Dimensions: ", images.shape)
    print("Label Dimensions: ", labels.shape)
    break

Extracting data/cifar-10-python.tar.gz to data
Image Dimensions:  torch.Size([128, 3, 32, 32])
Label Dimensions:  torch.Size([128])


In [4]:
###############
# Model
###############

class LeNet(nn.Module):
    def __init__(self, num_classes, grayscale = False):
        super(LeNet, self).__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes
        
        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3
        
        self.features = nn.Sequential(
                nn.Conv2d(in_channels = in_channels, out_channels = 6*in_channels, kernel_size = 5),
                nn.Tanh(),
                nn.MaxPool2d(kernel_size = 2),
                nn.Conv2d(in_channels = 6*in_channels, out_channels = 16*in_channels, kernel_size = 5),
                nn.Tanh(),
                nn.MaxPool2d(kernel_size = 2)
        )
        
        self.classifier = nn.Sequential(
                nn.Linear(16*5*5*in_channels, 120*in_channels),
                nn.Tanh(),
                nn.Linear(120*in_channels, 84*in_channels),
                nn.Tanh(),
                nn.Linear(84*in_channels, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim = 1)
        return logits, probas

In [5]:
torch.manual_seed(random_seed)

model = LeNet(num_classes, grayscale)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [6]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
        
        features = features.to(device)
        targets = targets.to(device)
        
        logits, probas = model(features)
        
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    
    return correct_pred.float() / num_examples * 100

start_time = time.time()
for epoch in range(num_epochs):
    
    model.train()
    
    for i, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)
        
        # Forward and Backprop
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        # Update model parameters
        optimizer.step()
        
        # LOGGING
        if not i % 50:
            print("Epoch: %03d/%03d || Batch: %03d/%03d || Cost: %.4f" % (epoch+1, num_epochs, i, len(train_loader), cost))
        
    model.eval()
    with torch.set_grad_enabled(False):
        print("Epoch: %03d/%03d || Train: %.3f%%" % (epoch+1, num_epochs, compute_accuracy(model, train_loader, device)))
    
    print("Time Elapsed: %.2f min" % ((time.time()-start_time)/60))

print("Total Training Time: %.2f min" % ((time.time()-start_time)/60))

Epoch: 001/010 || Batch: 000/391 || Cost: 2.3068
Epoch: 001/010 || Batch: 050/391 || Cost: 1.8193
Epoch: 001/010 || Batch: 100/391 || Cost: 1.6463
Epoch: 001/010 || Batch: 150/391 || Cost: 1.5756
Epoch: 001/010 || Batch: 200/391 || Cost: 1.4029
Epoch: 001/010 || Batch: 250/391 || Cost: 1.3118
Epoch: 001/010 || Batch: 300/391 || Cost: 1.3301
Epoch: 001/010 || Batch: 350/391 || Cost: 1.2877
Epoch: 001/010 || Train: 54.556%
Time Elapsed: 0.23 min
Epoch: 002/010 || Batch: 000/391 || Cost: 1.4057
Epoch: 002/010 || Batch: 050/391 || Cost: 1.2842
Epoch: 002/010 || Batch: 100/391 || Cost: 1.2604
Epoch: 002/010 || Batch: 150/391 || Cost: 1.3171
Epoch: 002/010 || Batch: 200/391 || Cost: 1.2911
Epoch: 002/010 || Batch: 250/391 || Cost: 1.3559
Epoch: 002/010 || Batch: 300/391 || Cost: 1.2006
Epoch: 002/010 || Batch: 350/391 || Cost: 1.1475
Epoch: 002/010 || Train: 60.400%
Time Elapsed: 0.45 min
Epoch: 003/010 || Batch: 000/391 || Cost: 1.1226
Epoch: 003/010 || Batch: 050/391 || Cost: 1.1907
Epoch:

In [ ]:
with torch.set_grad_enabled(False):
    print("Test Accuracy: %.2f%%" % (compute_accuracy(model, test_loader, device)))